In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/My Drive/MSc_Project_Colab/BAD_PyTorch/')

In [3]:
!pip install torchaudio

     |████████████████████████████████| 1.9 MB 5.3 MB/s 


In [4]:
import torch
import torchaudio
from torch import nn

from dcasedatasetcpu import DCASE_Dataset
from cnnbinary import CNNNetwork
from train_binary import ANNOTATIONS_FILE, AUDIO_DIR, SAMPLE_RATE, DURATION, NUM_SAMPLES

# ANNOTATIONS_FILE = '/content/drive/My Drive/DCASE_Datasets/labels/mini_metadata.csv'
# AUDIO_DIR = '/content/drive/My Drive/DCASE_Datasets/audio/'
# SAMPLE_RATE = 22050
# DURATION = 10
# NUM_SAMPLES = 22050 * DURATION


class_mapping = [
    "no-bird",
    "bird"
]


def predict(model, input, target, class_mapping):
    model.eval()
    with torch.no_grad():
        predictions = model(input).cuda()
        sigmoid = nn.Sigmoid()
        predictions = sigmoid(predictions)
        print(predictions)
        # Tensor (1, 10) -> [ [0.1, 0.01, ..., 0.6] ]
        if predictions[0] > 0.5:

          # predicted_index = predictions[0].argmax(0)
          predicted_index = 1
        elif predictions[0] < 0.5:
          predicted_index = 0
        print(predicted_index)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predicted, expected


if __name__ == "__main__":
    # load back the model
    cnn = CNNNetwork()
    state_dict = torch.load("/content/drive/My Drive/MSc_Project_Colab/BAD_PyTorch/cnn.pth", map_location=torch.device('cpu'))
    cnn.load_state_dict(state_dict)

    # load DCASE dataset
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

    dcase = DCASE_Dataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            "cpu")

    # get a sample from the dcase dataset for inference
    index = 0
    while index < 100:
      input, target = dcase[index][0], dcase[index][1]
      input.unsqueeze_(0)
      index += 1

    # make an inference
      predicted, expected = predict(cnn, input, target,
                                  class_mapping)
      print(f"Predicted: '{predicted}', expected: '{expected}'")
      print()


/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/64486.wav
/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/64486.wav


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[5.7653e-31]], device='cuda:0')
0
Predicted: 'no-bird', expected: 'no-bird'

/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/2525.wav
/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/2525.wav
tensor([[4.3495e-23]], device='cuda:0')
0
Predicted: 'no-bird', expected: 'no-bird'

/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/44981.wav
/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/44981.wav
tensor([[0.0023]], device='cuda:0')
0
Predicted: 'no-bird', expected: 'no-bird'

/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/101323.wav
/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/101323.wav
tensor([[8.8092e-06]], device='cuda:0')
0
Predicted: 'no-bird', expected: 'no-bird'

/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/165746.wav
/content/drive/My Drive/DCASE_Datasets/audio/ff1010bird/165746.wav
tensor([[0.1209]], device='cuda:0')
0
Predicted: 'no-bird', expected: 'no-bird'

/content/drive/My Drive/DCASE_Datasets/audio/ff1010bi